In [1]:
import re
import pandas as pd

#  读取 CSV
df = pd.read_csv('hotel.csv', encoding='utf-8')

#  重命名关键列
df.rename(columns={
    'タイトル':     'name',              
    'タイトルURL':  'url',                
    'タイトル2':    'location',           
    '価格7':        'price_total',        
    '価格8':        'price_per_person',   
    '概要':         'rating' ,
    'タイトル12':    'walk_time'
   
}, inplace=True)

In [2]:
#  提取 hotel_id（URL 中的 yad123456）
def extract_hotel_id(link):
    m = re.search(r'yad\d+', str(link))
    return m.group(0) if m else pd.NA

df['hotel_id'] = df['url'].apply(extract_hotel_id)

In [3]:
#  解析 location 拆分为 prefecture / city
#    示例 "石川県 > 金沢・湯涌"
loc_split = df['location'].str.split(' > ', expand=True)
df['prefecture'] = loc_split[0]
df['city']       = loc_split[1]


In [4]:
#  价格字段清洗：去除日元符号、逗号，“～”等，提取数字
for col in ['price_total', 'price_per_person']:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r'[¥,～\s名名人大人]+', '', regex=True)  # 去掉 ¥ , ～ 及文字
        .str.extract(r'(\d+)')                               # 抽出数字
        .astype(float)
    )

In [5]:
def extract_walk_time(access_str):
    s = str(access_str)
    # 优先匹配“徒歩xx分”
    m = re.search(r'徒歩\s*(\d+)\s*分', s)
    if not m:
        # 如果没有“徒歩”，也尝试匹配“xx分”
        m = re.search(r'(\d+)\s*分', s)
    return int(m.group(1)) if m else pd.NA

# 3. 应用到 access 列，生成新列
df['station_walk_time'] = df['walk_time'].apply(extract_walk_time)

# 4. 查看新增列效果
print(df[['walk_time', 'station_walk_time']].head())

                                           walk_time station_walk_time
0   金沢駅徒歩10分。または北鉄バス「武蔵ヶ辻・近江市場前」バス停下車目の前。コンビニまで徒歩約3分                10
1   金沢駅東口③・⑦～⑪番乗り場より「武蔵ヶ辻・近江町市場」バス停より徒歩4分／無料シャトルバスあり                 4
2                   JR金沢駅兼六園口（東口）から徒歩4分◇アクセス抜群！好立地☆彡                 4
3              JR金沢駅より北鉄バス乗車、「武蔵ケ辻・近江町市場」バス停下車徒歩約2分。                 2
4  JR金沢駅からバス「南町・尾山神社」迄7分下車1分/小松空港からバス「武蔵ヶ辻・近江町市場」...                 7


In [6]:
#  丢弃关键列缺失的行
df.dropna(subset=['hotel_id','name'], inplace=True)


In [7]:
# 删除重复 hotel_id，只保留第一条
df.drop_duplicates(subset='hotel_id', keep='first', inplace=True)


In [8]:

#  重建索引
df.reset_index(drop=True, inplace=True)


In [10]:

# 只保留关键列
key_cols = [
    'hotel_id',
    'name',
    'prefecture',
    'city',
    'price_per_person',
    'rating',
    'station_walk_time'
]
df = df[key_cols]

#  检查前几行
print(df[['hotel_id','name','prefecture','city','price_per_person','rating','station_walk_time']].head())

# 14. 再次检查
print(df.head())



    hotel_id                                    name prefecture   city  \
0  yad378719                        FORZA　ホテルフォルツァ金沢        石川県  金沢・湯涌   
1  yad350199  THE HOTEL SANRAKU KANAZAWA（ザ ホテル山楽 金沢）        石川県  金沢・湯涌   
2  yad304736                            ホテル・トリフィート金沢        石川県  金沢・湯涌   
3  yad388181          天然温泉 加賀の宝泉 御宿 野乃金沢（ドーミーインチェーン）        石川県  金沢・湯涌   
4  yad362769                            ホテルインターゲート金沢        石川県  金沢・湯涌   

   price_per_person  rating station_walk_time  
0           15600.0     4.6                10  
1          112000.0     4.8                 4  
2           16250.0     4.6                 4  
3           18450.0     4.5                 2  
4           17400.0     4.5                 7  
    hotel_id                                    name prefecture   city  \
0  yad378719                        FORZA　ホテルフォルツァ金沢        石川県  金沢・湯涌   
1  yad350199  THE HOTEL SANRAKU KANAZAWA（ザ ホテル山楽 金沢）        石川県  金沢・湯涌   
2  yad304736                            ホテル・ト

In [11]:

#  保存前处理结果
df.to_csv('hotel_cleaned.csv', index=False, encoding='utf-8-sig')